In [2]:
!pip install selenium

# hi

In [3]:
from bs4 import BeautifulSoup
import requests
import time, os
import pandas as pd
import numpy as np

In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = r"C:\ProgramData\Microsoft\Windows\Start Menu\Programs\chromedriver_win32/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [14]:

Boyoot = "https://www.bayut.sa/en/riyadh-region/properties-for-sale-in-riyadh/"
driver = webdriver.Chrome(chromedriver)
driver.get(Boyoot)
time.sleep(1)
    

In [6]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)#sleep_between_interactions

In [8]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [7]:

Location= soup.find_all('div', {"class":"_7afabd84"})
prices_scraping = soup.find_all('span', {"class":"f343d9ce"})
estate_type=soup.find_all('div',{"class":"_9a4e3964"})
baths=soup.find_all('span',{"aria-label":"Baths"})
rooms=soup.find_all('span',{"aria-label":"Beds"})
Area=soup.find_all('span',{"aria-label":"Area"}) 


In [8]:
import re

In [89]:
neighborhood =[]
direction=[]
region=[]
prices=[]
EstateType=[]
square_feet=[]
rooms_number=[]
baths_number=[]
p=1
while True:
    
    if p==1:
        url="https://www.bayut.sa/en/riyadh-region/properties-for-sale-in-riyadh/"
    else:
        url=f"https://www.bayut.sa/en/riyadh-region/properties-for-sale-in-riyadh/page-{p}/"
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    a=soup.find_all("div",{"class":"d6e81fd0"})
    
    
    for item in a:
        
        Location= item.find('div', {"class":"_7afabd84"})
        prices_scraping = item.find('span', {"class":"f343d9ce"})
        estate_type=item.find('div',{"class":"_9a4e3964"})
        baths=item.find('span',{"aria-label":"Baths"})
        rooms=item.find('span',{"aria-label":"Beds"})
        Area=item.find('span',{"aria-label":"Area"}) 

        EstateType.append(estate_type.text)
        try:
            prices.append(prices_scraping.text)
        except:
            prices.append(np.nan)
            
            
        try:
           
            baths_number.append(baths.text)
            rooms_number.append(rooms.text)

        except:
            baths_number.append(np.nan)
            rooms_number.append(np.nan)
            
        try:
            square_feet.append(Area.text.replace(' Sq. M.',''))
        except:
            square_feet.append(np.nan)
        L=Location.text
        split_location=L.split(',')
        try:
            if len(split_location)==4:
                neighborhood.append(split_location[0])
                direction.append(split_location[1])
                region.append(split_location[2])
            else:
                neighborhood.append(split_location[1])
                direction.append(split_location[2])
                region.append(split_location[3])
        except:
            continue


    p+=1
    if p==90:
        break;
    

In [90]:
df = pd.DataFrame(list(zip(neighborhood, direction,region,EstateType,square_feet, rooms_number, baths_number,prices )),
               columns =['neighborhood', 'direction','region','RealEstateType','square_feet', 'rooms_number', 'baths_number','prices'])
df

,neighborhood,direction,region,RealEstateType,square_feet,rooms_number,baths_number,prices
0,Al Ghadir,North Riyadh,Riyadh,Residential Building,"1,225",NaN,NaN,"33,000,000"
1,Al Narjis,North Riyadh,Riyadh,Villa,660,4,5,"5,200,000"
2,Al Qirawan,North Riyadh,Riyadh,Villa,718,6,8,"6,300,000"
3,Al Nuzhah,North Riyadh,Riyadh,Villa,450,4,5,"2,500,000"
4,Al Rabi,North Riyadh,Riyadh,Villa,550,4,5,"3,300,000"
...,...,...,...,...,...,...,...,...
2125,Laban,North Riyadh,Riyadh,Villa,450,4,6,"1,700,000"
2126,Al Mohammadiyah,North Riyadh,Riyadh,Villa,324,4,NaN,"2,950,000"
2127,Al Ghadir,West Riyadh,Riyadh,Residential Building,"1,200",NaN,6,"15,000,000"
2128,Dhahrat Laban,North Riyadh,Riyadh,Apartment,125,3,3,"395,000"


In [110]:
df.to_csv(r"C:\Users\x1doo\OneDrive\Documents\realstate.csv", sep=',')

In [106]:
df.prices.dropna(how='any', inplace=True)


In [103]:
df.prices.isna().value_counts()

False    2003
True      127
Name: prices, dtype: int64

In [108]:
df.square_feet.dropna(inplace=True)

In [56]:
df["RealEstateType"].value_counts()

Villa                   1800
Apartment                218
Residential Land         144
Residential Building      95
Floor                     60
Palace                    30
Rest House                14
Hotel Apartment            5
Chalet                     3
Name: RealEstateType, dtype: int64

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2130 entries, 0 to 2129
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   neighborhood    2130 non-null   object
 1   direction       2130 non-null   object
 2   region          2130 non-null   object
 3   RealEstateType  2130 non-null   object
 4   square_feet     2123 non-null   object
 5   rooms_number    1465 non-null   object
 6   baths_number    1466 non-null   object
 7   prices          2003 non-null   object
dtypes: object(8)
memory usage: 133.2+ KB
